In [ ]:
import requests
import os
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm  # For the progress bar

# API Key
API_KEY = 'XXXX'

# Folder to save the data
output_folder = 'stock_data/eod'
os.makedirs(output_folder, exist_ok=True)

# URL to get the list of NASDAQ stocks
nasdaq_url = f'https://financialmodelingprep.com/stable/nasdaq-constituent?apikey={API_KEY}'

# URL to get historical EOD data for a stock
eod_url = 'https://financialmodelingprep.com/stable/historical-price-eod/full'

# Get the list of NASDAQ stock tickers
def get_nasdaq_tickers():
    response = requests.get(nasdaq_url)
    if response.status_code == 200:
        data = response.json()
        tickers = [stock['symbol'] for stock in data]
        return tickers
    else:
        print(f"Failed to fetch NASDAQ tickers. Status code: {response.status_code}")
        return []

# Save NASDAQ tickers to a CSV file
def save_tickers_to_csv(tickers):
    tickers_df = pd.DataFrame(tickers, columns=["ticker"])
    tickers_df.to_csv('nasdaq_tickers.csv', index=False)
    print(f"NASDAQ tickers saved to 'nasdaq_tickers.csv'.")

# Fetch historical data for each stock and save it as CSV
def fetch_and_save_eod_data(tickers, start_date, end_date):
    for ticker in tqdm(tickers, desc="Downloading EOD data", unit="ticker"):
        print(f"Downloading EOD data for {ticker}...")
        params = {
            'symbol': ticker,
            'apikey': API_KEY
        }

        # Fetch the EOD data for the stock
        response = requests.get(eod_url, params=params)
        if response.status_code == 200:
            data = response.json()

            # Check if the data is empty
            if data:
                # Convert the data into a DataFrame
                df = pd.DataFrame(data)

                # Filter data for the last 5 years
                df['date'] = pd.to_datetime(df['date'])
                df = df[df['date'] >= start_date]

                # Save to CSV file
                file_path = os.path.join(output_folder, f'{ticker}.csv')
                df.to_csv(file_path, index=False)
                print(f"Data for {ticker} saved to {file_path}")
            else:
                print(f"No data found for {ticker}")
        else:
            print(f"Failed to fetch EOD data for {ticker}. Status code: {response.status_code}")

# Set the date range (last 5 years)
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=5*365)).strftime('%Y-%m-%d')

# Get the NASDAQ stock tickers
tickers = get_nasdaq_tickers()

# Save NASDAQ tickers to CSV
save_tickers_to_csv(tickers)

# Fetch and save the EOD data for each stock with a progress bar
fetch_and_save_eod_data(tickers, start_date, end_date)


In [7]:
# train_cgan.py — Optimized cGAN Training with Full GPU Utilization

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm, trange
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers
import joblib
import time

# === Mixed Precision ===
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# === Config ===
SEQ_IN = 250
SEQ_OUT_LIST = [1, 30, 90, 250]
z_dim = 100
batch_size = 1024  # Maximize batch size for GPU
lr = 1e-4

# === Data Loading ===
data_path = 'stock_data/eod'
tickers = pd.read_csv('nasdaq_tickers.csv')['ticker'].tolist()

X_cond, Y_target = [], []

for ticker in tqdm(tickers, desc="Slicing each stock"):
    filepath = os.path.join(data_path, f"{ticker}.csv")
    if not os.path.isfile(filepath):
        continue

    df = pd.read_csv(filepath, parse_dates=['date'], index_col='date')
    if 'close' not in df.columns or df['close'].isna().sum() > 0:
        continue

    log_returns = np.log(df['close'] / df['close'].shift(1)).dropna().values

    for horizon in SEQ_OUT_LIST:
        for i in range(len(log_returns) - (SEQ_IN + horizon)):
            cond = log_returns[i:i+SEQ_IN]
            future = log_returns[i+SEQ_IN:i+SEQ_IN+horizon]
            padded_future = np.pad(future, (0, max(SEQ_OUT_LIST) - horizon))
            X_cond.append(cond)
            Y_target.append(padded_future)

X_cond = np.array(X_cond, dtype=np.float32)
Y_target = np.array(Y_target, dtype=np.float32)

if X_cond.shape[0] == 0:
    raise ValueError("❌ No training samples generated! Check SEQ_IN, data size.")

print(f"✅ Training samples: {X_cond.shape[0]}")

# === Normalize ===
scaler_X = StandardScaler()
scaler_Y = StandardScaler()

X_cond_scaled = scaler_X.fit_transform(X_cond).astype(np.float32)
Y_target_scaled = scaler_Y.fit_transform(Y_target).astype(np.float32)

# === Build Models ===
def build_generator(z_dim, cond_dim, output_dim):
    inputs = layers.Input(shape=(z_dim + cond_dim,))
    x = layers.Dense(2048, activation='relu')(inputs)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dense(output_dim, dtype='float32')(x)  # Ensure float32 output for compatibility
    return tf.keras.Model(inputs, x)

def build_discriminator(cond_dim, input_dim):
    inputs = layers.Input(shape=(cond_dim + input_dim,))
    x = layers.Dense(2048)(inputs)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dense(1024)(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dense(1, activation='sigmoid', dtype='float32')(x)
    return tf.keras.Model(inputs, x)

G = build_generator(z_dim, SEQ_IN, max(SEQ_OUT_LIST))
D = build_discriminator(SEQ_IN, max(SEQ_OUT_LIST))

# === Optimizers ===
bce = tf.keras.losses.BinaryCrossentropy()
g_opt = tf.keras.optimizers.Adam(learning_rate=lr)
d_opt = tf.keras.optimizers.Adam(learning_rate=lr)

# === Dataset ===
dataset = (
    tf.data.Dataset.from_tensor_slices((X_cond_scaled, Y_target_scaled))
    .shuffle(buffer_size=50000)
    .batch(batch_size, drop_remainder=False)
    .prefetch(tf.data.AUTOTUNE)
)

@tf.function
def train_step(condition, real_output):
    batch_size = tf.shape(condition)[0]
    noise = tf.random.normal((batch_size, z_dim))
    input_gen = tf.concat([noise, condition], axis=1)

    with tf.GradientTape() as d_tape, tf.GradientTape() as g_tape:
        fake_output = G(input_gen, training=True)
        real_input = tf.concat([condition, real_output], axis=1)
        fake_input = tf.concat([condition, fake_output], axis=1)

        d_real = D(real_input, training=True)
        d_fake = D(fake_input, training=True)

        d_loss = bce(tf.ones_like(d_real), d_real) + bce(tf.zeros_like(d_fake), d_fake)
        g_loss = bce(tf.ones_like(d_fake), d_fake)

    d_grads = d_tape.gradient(d_loss, D.trainable_variables)
    g_grads = g_tape.gradient(g_loss, G.trainable_variables)

    d_opt.apply_gradients(zip(d_grads, D.trainable_variables))
    g_opt.apply_gradients(zip(g_grads, G.trainable_variables))
    return d_loss, g_loss

# === Train ===
epochs = 3000
print("🚀 Training...")
for epoch in trange(epochs):
    for cond_batch, real_batch in dataset:
        d_loss, g_loss = train_step(cond_batch, real_batch)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: D = {d_loss.numpy():.4f} | G = {g_loss.numpy():.4f}")

# === Save Artifacts ===
os.makedirs("models", exist_ok=True)
G.save('models/generator_stockwise_cgan.h5')
D.save('models/discriminator_stockwise_cgan.h5')
joblib.dump(scaler_X, 'models/scaler_X.pkl')
joblib.dump(scaler_Y, 'models/scaler_Y.pkl')
print("✅ Models and scalers saved.")


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


2025-04-09 23:54:19.407086: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
Slicing each stock: 100%|██████████| 101/101 [00:07<00:00, 14.36it/s]


✅ Training samples: 354572
🚀 Training...


  0%|          | 0/3000 [00:00<?, ?it/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <gast.gast.Expr object at 0x7f098db51690>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <gast.gast.Expr object at 0x7f098db51690>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

2025-04-10 01:42:23.586459: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
 97%|█████████▋| 2901/3000 [1:51:34<03:41,  2.24s/it]

Epoch 2900: D = nan | G = nan


2025-04-10 01:46:07.834281: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
100%|██████████| 3000/3000 [1:55:16<00:00,  2.31s/it]

✅ Models and scalers saved.



/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
